In [7]:
import dspy

dspy.LM(model="dolphin-phi:2.7b")
dspy.configure(lm=lm)


AttributeError: module 'dspy' has no attribute 'LM'

In [2]:
from litellm import completion

response = completion(
    model="ollama_chat/dolphin-phi:2.7b", 
    messages=[{ "content": "respond in 20 words. who are you?","role": "user"}], 
)
print(response)

ModelResponse(id='chatcmpl-9484c266-d2b0-48ca-8a28-186c4cb6162e', created=1752787670, model='ollama_chat/dolphin-phi:2.7b', object='chat.completion', system_fingerprint=None, choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hello! I am an artificial intelligence designed to assist with various tasks and provide information when needed. I can help you with your queries, solve problems, or simply have a conversation with you. My aim is to be helpful and make your life easier. How may I be of service to you today?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields=None))], usage=Usage(completion_tokens=61, prompt_tokens=33, total_tokens=94, completion_tokens_details=None, prompt_tokens_details=None))


In [6]:
import dspy
from litellm import completion
from typing import List, Dict, Any

class LiteLLMLM(dspy.LM):
    """Custom DSPy Language Model using LiteLLM"""
    
    def __init__(self, model: str = "ollama/dolphin-phi:2.7b", **kwargs):
        self.model = model
        self.kwargs = kwargs
        super().__init__(model)
    
    def generate(self, prompt: str, **kwargs) -> List[str]:
        """Generate response using LiteLLM"""
        messages = [{"content": prompt, "role": "user"}]
        
        response = completion(
            model=self.model,
            messages=messages,
            **{**self.kwargs, **kwargs}
        )
        
        return [response.choices[0].message.content]
    
    def __call__(self, prompt: str, **kwargs) -> List[str]:
        """DSPy interface method"""
        return self.generate(prompt, **kwargs)

# Initialize the custom LM
lm = LiteLLMLM(model="ollama_chat/dolphin-phi:2.7b")

# Configure DSPy to use our custom LM
dspy.configure(lm=lm)

# Define a simple signature for your task
class BasicQA(dspy.Signature):
    """Answer questions concisely"""
    question = dspy.InputField()
    answer = dspy.OutputField(desc="A concise answer in about 20 words")

# Create Chain of Thought module
cot = dspy.ChainOfThought(BasicQA)

# Example usage
if __name__ == "__main__":
    # Use Chain of Thought reasoning
    response = cot(question="Who are you?")
    print(f"Answer: {response.answer}")
    
    # You can also access the reasoning chain if available
    if hasattr(response, 'rationale'):
        print(f"Reasoning: {response.rationale}")
    
    # Example with different question
    response2 = cot(question="What is the capital of France?")
    print(f"Answer: {response2.answer}")

AttributeError: module 'dspy' has no attribute 'LM'

In [3]:
# part 2
import dspy
import random

# Note: Run the Ollama configuration cell first!

class FactJudge(dspy.Signature):
    """Judge if the answer is factually correct based on the context."""
    context = dspy.InputField(desc="Context for the prediction")
    question = dspy.InputField(desc="Question to be answered")
    answer = dspy.InputField(desc="Answer for the question")
    factually_correct: bool = dspy.OutputField(desc="Is the answer factually correct based on the context?")

class QASystem(dspy.Signature):
    """Answer questions based on given context."""
    context = dspy.InputField(desc="Context to answer from")
    question = dspy.InputField(desc="Question to answer")
    answer = dspy.OutputField(desc="Answer to the question")

# Initialize the judge with Gemma 2 2b
judge = dspy.ChainOfThought(FactJudge)

# Initialize QA system with Gemma 2 2b
qa_system = dspy.ChainOfThought(QASystem)

def factuality_metric(example, pred):
    # Use the configured Gemma 2 2b model for judging
    factual = judge(context=example.context, question=example.question, answer=pred.answer)
    return factual.factually_correct

# Sample data for testing
sample_examples = [
    {
        "context": "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. It is named after the engineer Gustave Eiffel, whose company designed and built the tower. The tower is 330 metres (1,083 ft) tall, about the same height as an 81-storey building.",
        "question": "How tall is the Eiffel Tower?",
        "answer": "330 metres"
    },
    {
        "context": "Python is a high-level, interpreted programming language. It was created by Guido van Rossum and first released in 1991. Python's design philosophy emphasizes code readability with its notable use of significant whitespace.",
        "question": "Who created Python?",
        "answer": "Guido van Rossum"
    },
    {
        "context": "The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, wood, and other materials. It was built along the northern borders of China to protect against invasions. The wall stretches over 13,000 miles.",
        "question": "How long is the Great Wall of China?",
        "answer": "Over 13,000 miles"
    }
]

# Convert to dspy.Example objects
examples = [dspy.Example(**ex) for ex in sample_examples]

def run_qa_demo():
    print("=== DSPy Fact-Checking Demo ===")
    print("Using Gemma 2 2b for both QA and fact-checking\n")
    
    # Test with a new question
    test_context = "Albert Einstein was a German-born theoretical physicist who developed the theory of relativity. He received the Nobel Prize in Physics in 1921 for his explanation of the photoelectric effect."
    test_question = "What did Einstein win the Nobel Prize for?"
    
    print(f"Context: {test_context}")
    print(f"Question: {test_question}")
    
    # Generate answer using Gemma 2 2b
    prediction = qa_system(context=test_context, question=test_question)
    generated_answer = prediction.answer
    
    print(f"Generated Answer: {generated_answer}")
    
    # Check factuality using Gemma 2 2b
    fact_check = judge(
        context=test_context, 
        question=test_question, 
        answer=generated_answer
    )
    is_factual = fact_check.factually_correct
    reasoning = fact_check.reasoning if hasattr(fact_check, 'reasoning') else "No reasoning provided"
    
    print(f"Factually Correct: {is_factual}")
    print(f"Reasoning: {reasoning}")
    
    # Test the metric function with sample examples
    print("\n=== Testing Factuality Metric ===")
    for i, example in enumerate(examples[:2]):  # Test first 2 examples
        print(f"\nExample {i+1}:")
        print(f"Question: {example.question}")
        print(f"Expected Answer: {example.answer}")
        
        # Create a prediction object
        pred = dspy.Example(answer=example.answer)
        
        # Test the metric
        is_factual = factuality_metric(example, pred)
        print(f"Factually Correct: {is_factual}")

def test_with_wrong_answer():
    print("\n=== Testing with Incorrect Answer ===")
    wrong_example = dspy.Example(
        context="The capital of France is Paris. Paris is located in the north-central part of France.",
        question="What is the capital of France?",
        answer="London"  # Wrong answer
    )
    
    pred = dspy.Example(answer="London")
    is_factual = factuality_metric(wrong_example, pred)
    print(f"Question: {wrong_example.question}")
    print(f"Wrong Answer: London")
    print(f"Factually Correct: {is_factual}")

if __name__ == "__main__":
    try:
        run_qa_demo()
        test_with_wrong_answer()
    except Exception as e:
        print(f"Error: {e}")
        print("Make sure you have:")
        print("1. Run the Ollama configuration cell first")
        print("2. Ollama running: ollama serve")
        print("3. Gemma 2 2b model: ollama pull gemma2:2b")

AttributeError: module 'dspy' has no attribute 'Signature'